<a href="https://colab.research.google.com/github/rodrigobernall/ds4a_group_30_FINAL_PROJECT/blob/master/scripts/data_wrangling/SECOP_I_2019_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

More SECOP data can be found [here](https://www.datos.gov.co/Ciencia-Tecnolog-a-e-Innovaci-n/Inventario-de-Datasets/2irh-ijg2).

In [0]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import urllib.request
import json
from pandas.io.json import json_normalize

# Importing the data

We download the 2019 data from a Dropbox URL (zip file.)

In [0]:
url = 'https://www.dropbox.com/s/r56zkj70r5eldmn/SI2019.zip?dl=1'
print('Beginning file download with urllib2...')
urllib.request.urlretrieve(url, 'SI2019.zip')

Extracting the data

In [22]:
!ls

sample_data  SI2019.json  SI2019.zip


In [0]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('SI2019.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

Let's read the data into Pandas using `pd.read_json()`

In [23]:
si2019 = pd.read_json('SI2019.zip', encoding='latin-1', lines=True)
si2019.head()

,Release
0,"{'ocid': 'ocds-k50g02-19-12-8923103', 'id': '1..."
1,"{'ocid': 'ocds-k50g02-19-12-8923107', 'id': '1..."
2,"{'ocid': 'ocds-k50g02-19-4-9099228', 'id': '19..."
3,"{'ocid': 'ocds-k50g02-19-12-8923112', 'id': '1..."
4,"{'ocid': 'ocds-k50g02-19-12-8923114', 'id': '1..."


Now let's extract just one record, the first one:

In [0]:
df = si2019.head(1).copy()
df

Let's explore its structure (Note: This is a preview. In the actual file, non-standard characters are kept as they are, ie., they are not converted to Unicode strings):

In [25]:
parsed = json.loads(df['Release'].to_json())
print(json.dumps(parsed, indent=4, sort_keys=False))

{
    "0": {
        "ocid": "ocds-k50g02-19-12-8923103",
        "id": "19-12-8923103",
        "date": "2019-01-25T10:00:30.000Z",
        "tag": [
            "contract"
        ],
        "initiationType": "tender",
        "parties": [
            {
                "name": "META - ALCALD\u00cdA MUNICIPIO DE MESETAS",
                "id": "891502397",
                "identifier": {
                    "scheme": "CO-RUE",
                    "id": "891502397",
                    "legalName": "META - ALCALD\u00cdA MUNICIPIO DE MESETAS"
                },
                "address": {
                    "locality": "Mesetas",
                    "region": "Meta",
                    "countryName": "COLOMBIA"
                },
                "contactPoint": {
                    "name": "LILIANA CASTRO",
                    "email": "bancodeproyectos@mesetas-meta.gov.co",
                    "telephone": "(8) 6598012"
                },
                "details": {
               

# Parsing the JSON records to obtain usable dataframes

We shall create a function that does the following *for each record*:

1. Normalises the JSON file.
1. Checks which columns are JSON arrays (Python lists).
1. For each column that is an array, it normalises it.
1. Repeat the previous steps until there are no arrays left.
1. Keeps track of the identifiers for each column.
1. Creates a normalised DataFrame with all the data.


This function is called `json_row_to_df()`.

Some good resources are [this one](https://mindtrove.info/flatten-nested-json-with-pandas/), [this one](https://stackoverflow.com/questions/45418334/using-pandas-json-normalize-on-nested-json-with-arrays), [this one](https://stackoverflow.com/questions/45672130/how-to-identify-a-pandas-column-is-a-list) and [this one](https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe/53831756#53831756).

The function's pseudocode is the following (notice the use of recursion):

```
def json_row_to_df(record):

    if this level is a Json dictionary:
        normalise it
    elif this level is a Json array:
        extract its first element
        json_row_to_df(first element)
    elif this level is a 1-dimension element:
        merge it with the upper level (possibly normalised) dataframe
    
    return merged dataframe
```



In [26]:
RELEASE = df['Release']
MAIN = json_normalize(RELEASE).set_index('ocid')
ocid = MAIN.index[0]

## Las ramas primarias
PARTIES = json_normalize(RELEASE, record_path='parties')
MAIN = MAIN.drop(columns=['parties'])
PARTIES['ocid'] = ocid
PARTIES = PARTIES.set_index('ocid')
PARTIES = PARTIES.add_prefix('PARTIES.')

CONTRACTS = json_normalize(RELEASE, record_path='contracts')
MAIN = MAIN.drop(columns=['contracts'])
CONTRACTS['ocid'] = ocid
CONTRACTS = CONTRACTS.set_index('ocid')
CONTRACTS = CONTRACTS.add_prefix('CONTRACTS.')

## Ramas secundarias

PLANNING_MILESTONES = json_normalize(MAIN['planning.milestones'][0])
MAIN = MAIN.drop(columns=['planning.milestones'])
PLANNING_MILESTONES['ocid'] = ocid
PLANNING_MILESTONES = PLANNING_MILESTONES.set_index('ocid')
PLANNING_MILESTONES = PLANNING_MILESTONES.add_prefix('PLANNING.MILESTONES.')

ITEMS = json_normalize(CONTRACTS['CONTRACTS.items'][0])
CONTRACTS = CONTRACTS.drop(columns=['CONTRACTS.items'])
ITEMS['ocid'] = ocid
ITEMS = ITEMS.set_index('ocid')
ITEMS = ITEMS.add_prefix('CONTRACTS.ITEMS.')

ADDITIONALCLASSIFICATIONS = json_normalize(ITEMS['CONTRACTS.ITEMS.additionalClassifications'][0])
ITEMS = ITEMS.drop(columns=['CONTRACTS.ITEMS.additionalClassifications'])
ADDITIONALCLASSIFICATIONS['ocid'] = ocid
ADDITIONALCLASSIFICATIONS = ADDITIONALCLASSIFICATIONS.set_index('ocid')
ADDITIONALCLASSIFICATIONS = ADDITIONALCLASSIFICATIONS.add_prefix('CONTRACTS.ITEMS.ADDITIONALCLASSIFICATIONS.')

# CONTRACTS_PERIOD = json_normalize(CONTRACTS['CONTRACTS.period'][0])
# CONTRACTS = CONTRACTS.drop(columns=['CONTRACTS.period'])
# CONTRACTS_PERIOD['ocid'] = ocid
# CONTRACTS_PERIOD = CONTRACTS_PERIOD.set_index('ocid')
# CONTRACTS_PERIOD = CONTRACTS_PERIOD.add_prefix('CONTRACTS.PERIOD.')

MAIN
# ## JOIN

# RESULT = MAIN.join(CONTRACTS).join(ITEMS).join(ADDITIONALCLASSIFICATIONS).join(PARTIES).join(PLANNING_MILESTONES).join(CONTRACTS_VALUE).join(CONTRACTS_PERIOD)
# RESULT = pd.DataFrame(RESULT.stack()).reset_index()
# #RESULT.index = 
# i = RESULT['level_1'].str.split('.',expand=True)
# v = RESULT.values
# #pd.DataFrame(v, index=[i])
# #CONTRACTS
# i.set_index([0,1,2,3])
# #MAIN

,id,date,tag,initiationType,buyer.id,buyer.name,planning.rationale,planning.budget.id,planning.budget.description,planning.budget.amount.amount,planning.budget.amount.currency,tender.id,tender.title,tender.description,tender.status,tender.procuringEntity.id,tender.procuringEntity.name,tender.items,tender.value.amount,tender.value.currency,tender.procurementMethod,tender.procurementMethodDetails,tender.procurementMethodDetailsRationale,tender.submissionMethod,tender.submissionMethodDetails,tender.expressionAddress.countryName,tender.tenderPeriod.startDate,tender.milestones,tender.coveredBy
ocid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ocds-k50g02-19-12-8923103,19-12-8923103,2019-01-25T10:00:30.000Z,[contract],tender,891502397,META - ALCALDÍA MUNICIPIO DE MESETAS,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,20190000024,CDP,16800000.0,COP,19-12-8923103,CD - 043 DE 2019,PRESTACION DE SERVICIOS DE APOYO A LA GESTION ...,complete,891502397,META - ALCALDÍA MUNICIPIO DE MESETAS,"[{'id': '801117', 'description': 'Reclutamient...",16800000.0,COP,limited,Contratación Directa (Ley 1150 de 2007),Prestación de Servicios Profesionales y de Apo...,[inPerson],Municipio obtención: Mesetas Municipio entrega...,Colombia,2019-01-22T20:00:00.000Z,"[{'dueDate': '2019-01-22T20:00:00.000Z', 'id':...",[x_Estatuto_General_de_Contratación]
